# data

source : https://github.com/naver/nlp-challenge/blob/master/missions/ner/data/train/train_data

In [15]:
import pandas as pd

In [16]:
# data = pd.read_csv('data/train_data.txt', sep='\t',  header=None)
# data.columns = ['index' , 'content', 'tag']
# data['word_id'] = [k for k in range(len(data['content']))]

In [17]:
# data = pd.read_csv('data/indexingFile.csv', sep=',',  header=0)
# data.columns = ['number', 'index' , 'content', 'tag']
# data = data.drop(['number'], axis=1)
# data['word_id'] = [k for k in range(len(data['content']))]

In [18]:
data = pd.read_csv('data/indexingFile_527.csv', sep=',',  header=0)
data.columns = ['number', 'index' , 'content', 'tag']
data = data.drop(['number'], axis=1)
data['word_id'] = [k for k in range(len(data['content']))]

In [19]:
data.head(25)

,index,content,tag,word_id
0,1,강사,1,0
1,2,로,0,1
2,3,활동,0,2
3,4,잘할,0,3
4,5,같은,0,4
5,1,일타,0,5
6,2,강사,1,6
7,3,로,0,7
8,4,같은,0,8
9,1,일타,0,9


To split by a sentence

In [20]:
content_split = []
tag_split = []
word_id_split = []

content_spliter = []
tag_spliter = []
word_spliter = []

for i in range(len(data)):
    content_spliter.append(data['content'].iloc[i])
    tag_spliter.append(data['tag'].iloc[i])
    word_spliter.append(data['word_id'].iloc[i])
    if (i == len(data)-1) : # the last word in a data
        content_split.append(content_spliter)
        content_spliter = []
        tag_split.append(tag_spliter)
        tag_spliter = []
        word_id_split.append(word_spliter)
        word_spliter = []
    elif (data['index'].iloc[i+1] == 1) : # the last word in a sentence
        content_split.append(content_spliter)
        content_spliter = []
        tag_split.append(tag_spliter)
        tag_spliter = []
        word_id_split.append(word_spliter)
        word_spliter = []
        
# sanity check
len(content_split) == len(tag_split) == len(word_id_split)

True

In [21]:
len(content_split)

703464

In [22]:
len(tag_split)

703464

In [23]:
len(word_id_split)

703464

## vocabulary

konlpy tokenizer

In [24]:
from konlpy.tag import Okt
okt = Okt()

print('형태소 분석 :',okt.morphs("자연어 처리 연구를 기반으로 인공지능 모델을 구성하고 있습니다."))
print('품사 태깅 :',okt.pos("자연어 처리 연구를 기반으로 인공지능 모델을 구성하고 있습니다."))
print('명사 추출 :',okt.nouns("자연어 처리 연구를 기반으로 인공지능 모델을 구성하고 있습니다."))

형태소 분석 : ['자연어', '처리', '연구', '를', '기반', '으로', '인공', '지능', '모델', '을', '구성', '하고', '있습니다', '.']
품사 태깅 : [('자연어', 'Noun'), ('처리', 'Noun'), ('연구', 'Noun'), ('를', 'Josa'), ('기반', 'Noun'), ('으로', 'Josa'), ('인공', 'Noun'), ('지능', 'Noun'), ('모델', 'Noun'), ('을', 'Josa'), ('구성', 'Noun'), ('하고', 'Josa'), ('있습니다', 'Adjective'), ('.', 'Punctuation')]
명사 추출 : ['자연어', '처리', '연구', '기반', '인공', '지능', '모델', '구성']


To create the vocabulary corpus

In [25]:
# to select unique value in content_split
unique_content_list = list(set([x for sublist in content_split for x in sublist]))

# to select only string
filtered = filter(lambda voca: isinstance(voca, str), unique_content_list)
unique_filtered_content_list = list(filtered)

# concatenation for tokenizer
unique_filtered_content_str = ' '.join(unique_filtered_content_list)
tokenizer_voca = okt.nouns(unique_filtered_content_str)
print(len(tokenizer_voca))
print(tokenizer_voca[:10])

75559
['세트', '해하', '안슬것', '별식', '좆', '급', '중', '놈', '이기', '개비']


In [26]:
len(okt.nouns(unique_filtered_content_str))

75559

In [27]:
tokenizer_voca = okt.nouns(unique_filtered_content_str)

In [28]:
tokenizer_voca[:10]

['세트', '해하', '안슬것', '별식', '좆', '급', '중', '놈', '이기', '개비']

In [29]:
voca2idex = {gene: idx+5 for idx, gene in enumerate(tokenizer_voca)}
voca2idex['<unk>']= 0 # unknown
voca2idex['<pad>']= 1 # padding
voca2idex['<bos>']= 2 # begining of sentence
voca2idex['<eos>']= 3 # end of sentence
voca2idex['<mask>']= 4 # masking

# configuration parameters
VOCA_SIZE = len(voca2idex)
print(VOCA_SIZE)

# index2voca
idex2voca = {v:k for k,v in voca2idex.items()}
#print(list(gene2idex.keys())[list(gene2idex.values()).index(16)]) 
print(f'{idex2voca[0]}')

21062
<unk>


voca to index

In [30]:
# seq_to_index function
def sequence_to_index(string_list, voca2idex):
    '''
    INPUT
        string_list : string list
    OUTPUT
        integer list
    '''
    converted_index = [voca2idex[word] if word in voca2idex.keys() else voca2idex['<unk>'] for word in string_list]
    return converted_index

# example
print(sequence_to_index(['멋대로', '가나다라마'], voca2idex)) # example

[60453, 0]


## tag integer encodding

In [31]:
tag_list = data['tag'].unique()
num_tags = len(tag_list)
tag_list

array([1, 0])

In [32]:
# tag2idex = {tag: idx for idx, tag in enumerate(tag_list)}
# tag2idex

In [33]:
idex2tag = {}
idex2tag[0]='<unk>'
idex2tag[1]='<pad>'
idex2tag[2]='<bos>'
idex2tag[3]='<eos>'
idex2tag[4]='<mask>'
idex2tag[5]='non'
idex2tag[6]='coffe'

In [34]:
tag2idex = {'<unk>':0,'<pad>':1,'<bos>':2,'<eos>':3,'<mask>':4, 0:5, 1:6}

In [35]:
tag2idex[0]

5

In [37]:
# seq_to_index function
def tag_to_index(sequence, embedding_index):
    '''
    INPUT
        sequence : string list
    OUTPUT
        integer list
    '''
    converted_index = [embedding_index[word] if word in embedding_index.keys() else embedding_index['<unk>'] for word in sequence]
    return converted_index

# # index2gene
# idex2tag = {v:k for k,v in tag2idex.items()}
# idex2tag[0]='[UNK]'
# idex2tag[1]='[PAD]'
# idex2tag[2]='[CLS]'
# idex2tag[3]='[SEP]'
# idex2tag[4]='[MASK]'
# #print(list(gene2idex.keys())[list(gene2idex.values()).index(16)]) 
# print(f'{idex2tag[1]}')

print(tag_to_index(['5', '6'], tag2idex)) # example

[0, 0]


In [ ]:
# tag_to_index(['PER_B', 'DAT_B'], tag2idex)

In [54]:
import pickle
with open("data/voca2idex.pickle", "wb") as fw:
    pickle.dump(voca2idex, fw)
with open("data/idex2voca.pickle", "wb") as fw:
    pickle.dump(idex2voca, fw)    
with open("data/tag2idex.pickle", "wb") as fw:
    pickle.dump(tag2idex, fw)
with open("data/idex2tag.pickle", "wb") as fw:
    pickle.dump(idex2tag, fw)

## tokenizer

In [ ]:
# content_tokenizer = []
# word_id_tokenizer = []
# tag_id_tokenizer = []
# label_ids_tokenizer = []
# for i in data['word_id'][0:50]:
#     # tokenizer
#     content_tokenizer += tokenizer.encode(data['content'].iloc[i])[1:-1]
#     word_id_tokenizer += [i]*(len(tokenizer.encode(data['content'].iloc[i]))-2)    
#     label_ids_tokenizer += [tag2idex[data['tag'].iloc[i]]] + [-100] * (len(tokenizer.encode(data['content'].iloc[i]))-3)
 
    
# data_tokenizer = pd.DataFrame({'token': content_tokenizer, 'word_id': word_id_tokenizer, 'label_ids': label_ids_tokenizer})

In [42]:
# tokenizer
tokenizer = okt.morphs

In [82]:
import numpy as np
import pandas as pd
def tokenizer_per_sentece(content_list, tag_list, word_id_list):
    """
    INPUT:
        content_split_list : a list that represents a sentence 
            e.g. ['비토리오', '양일', '만에', '영사관', '감호', '용퇴,', '항룡', '압력설', '의심만', '가율']
        tag_split_list
            e.g. ['PER_B', 'DAT_B', '-', 'ORG_B', 'CVL_B', '-', '-', '-', '-', '-']
        word_id_split_list
            e.g. [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    OUTPUT: tokenzier results
        content_tokenizer : a tokenized content list without special tokens
        word_id_tokenizer : a word id list
        tag_id_tokenizer : a tag id list that the only first content has a tag.
    """
    # sanity check
    if not (len(content_list)==len(tag_list)==len(word_id_list)):
        print("check the input data")
    
    # tokenizer
    content_tokenizer = []
    word_id_tokenizer = []
    tag_id_tokenizer = []
    
    # error handling : nan in input
    if any(pd.isna(content_list)):
        return [], [], []
    
    for i in range(len(word_id_list)):
        # tokenizer
        content_tokenizer += tokenizer(content_list[i]) # to remove special tokens [CLS] and [SEP]
        word_id_tokenizer += [word_id_list[i]] * (len(tokenizer(content_list[i])))
        tag_id_tokenizer += [tag2idex[tag_list[i]]] + [-100] * (len(tokenizer(content_list[i]))-1)
    
    # to convert string into index
    content_tokenizer = sequence_to_index(content_tokenizer, voca2idex)
    
    # error handling : tokenizer does not work properly
    if not (len(content_tokenizer) == len(word_id_tokenizer) == len(tag_id_tokenizer)):
        # tokenzier does not work properly becuase of noise.
        print(f'sanity check : {len(content_tokenizer) == len(word_id_tokenizer) == len(tag_id_tokenizer)}')
        content_tokenizer = []
        word_id_tokenizer = []
        tag_id_tokenizer = []
    return content_tokenizer, tag_id_tokenizer, word_id_tokenizer

In [61]:
content_split[0]

['강사', '로', '활동', '잘할', '같은']

In [83]:
a, b, c = tokenizer_per_sentece(content_split[0], tag_split[0], word_id_split[0])

In [84]:
a

[13782, 75328, 67477, 0, 13189, 0]

In [90]:
[idex2voca[word] for word in a]

['강사', '로', '활동', '<unk>', '할', '<unk>']

In [85]:
b

[6, 5, 5, 5, -100, 5]

In [86]:
c

[0, 1, 2, 3, 3, 4]

error case

In [39]:
content_split[172999]

['\ufeff', '가장', '한', '추출', '콜드', '추출', '에서', '카페인', '변화']

In [40]:
tag_split[172999]

[0, 0, 0, 1, 1, 1, 0, 1, 0]

In [41]:
word_id_split[172999]

[564651, 564652, 564653, 564654, 564655, 564656, 564657, 564658, 564659]

In [42]:
a, b, c = tokenizer_per_sentece(content_split[172999], tag_split[172999], word_id_split[172999])

sanity check : False


In [43]:
a

[]

Error case

In [66]:
content_split[500120]

[nan]

In [67]:
import numpy as np
np.isnan(content_split[500120])

array([ True])

In [68]:
np.isnan(content_split[500120])

array([ True])

In [69]:
type(content_split[500120][0])

float

In [70]:
a, b, c = tokenizer_per_sentece(content_split[500120], tag_split[500120], word_id_split[500120])

In [71]:
a

[]

error

In [72]:
content_split[598602]

['1', '1', 'coffee', nan, 'Mp']

In [73]:
pd.isna(content_split[598602])

array([False, False, False,  True, False])

In [74]:
any(pd.isna(content_split[598602]))

True

In [75]:
a, b, c = tokenizer_per_sentece(content_split[598602], tag_split[598602], word_id_split[598602])

In [91]:
from tqdm import tqdm 
def tokenizer_all_data(content_split, tag_split, word_id_split):
    # sanity check
    if not (len(content_split)==len(tag_split)==len(word_id_split)):
        print("check the input data")
        
    content_split_token, tag_split_token, word_id_split_token = [], [], []
    for i in tqdm(range(len((content_split)))):
        #print(i)
        content, tag, wordID = tokenizer_per_sentece(content_split[i], tag_split[i], word_id_split[i])
        content_split_token.append(content)
        tag_split_token.append(tag)
        word_id_split_token.append(wordID)
    return content_split_token, tag_split_token, word_id_split_token

In [92]:
content_split_tok, tag_split_tok, word_id_split_tok = tokenizer_all_data(content_split, tag_split, word_id_split)

100%|██████████████████████████████████| 703464/703464 [59:20<00:00, 197.58it/s]


record
- 20220527 : KoBert
- 20220531 : Bert

In [93]:
import pickle
with open("data/content_split_tok_20220531.pickle", "wb") as fw:
    pickle.dump(content_split_tok, fw)
with open("data/tag_split_tok_20220531.pickle", "wb") as fw:
    pickle.dump(tag_split_tok, fw)
with open("data/word_id_split_tok_20220531.pickle", "wb") as fw:
    pickle.dump(word_id_split_tok, fw)

In [65]:
import torch
import pickle
with open("data/content_split_tok_20220531.pickle", "rb") as fr:
    content_split_tok = pickle.load(fr)
with open("data/tag_split_tok_20220531.pickle", "rb") as fr:
    tag_split_tok = pickle.load(fr)
with open("data/word_id_split_tok_20220531.pickle", "rb") as fr:
    word_id_split_tok = pickle.load(fr)

In [53]:
#example
[tokenizer.decode(token) for token in content_split_tok[0]]

['강', '사', '하면', '잘', '할', '스타', '는', '?', '.']

Padding

In [54]:
print(max([len(x) for x in content_split_tok]))
print(sum([len(x) for x in content_split_tok])/len(content_split_tok))

653
6.924360502429632


In [94]:
def prep_for_tensor(token_list, max_len=512):
    '''
    This function get a list of token lists and converts a input list into a list in size of max_len including special tokens.
    
    INPUT
        token_list : a list of token lists
            e.g. [[1,2,3,4], [5,6,7,8], [9, 10, 11, 12, 13, 14, 15]]
        max_len : int
    OUTPUT
        token_ls : a list of lists in size of max_len with special tokens. The last token that does not fit the size would be dropped.
            e.g. [[0, 1, 2, 3, 4, 0], [0, 5, 6, 7, 8, 0], [0, 9, 10, 11, 12, 0]]
    '''
    # setting
    max_len = max_len-2 # to consider the special tokens
    pad = 1 # an index of special token
    bos = 2 # an index of special token
    eos = 3 # an index of special token
    token_output_ls = []
    token_ls = []
    #token_ls_iter_position = 0 # index of the last element in token_ls
    
    for index, tokens in enumerate(token_list):
        n_token = len(tokens)
        
        if len(token_ls) + n_token <= max_len :
            token_ls += tokens
            #token_ls_iter_position += len(token_ls)
        else : 
            cut_position = max_len - len(token_ls)
            #assert cut_position >= 0
            token_ls += tokens[:cut_position]

            # to collect the current token_ls
            token_ls = [bos] + token_ls + [eos] # to add special tokens

            if not len(token_ls)-2 == max_len : 
                print(f'index: {index}')
                print(f'cut_position: {cut_position}')
                print(len(token_ls))
            assert len(token_ls)-2 == max_len
            token_output_ls.append(token_ls)

            # to update token_ls for next iteration
            token_ls = tokens[cut_position:]
            #token_ls_iter_position = len(token_ls) - 1
            
            # if token_ls is longer than max_len, it requires an additional cleavage
            while (len(token_ls) > max_len):
                token_output_ls.append([bos] + token_ls[:max_len] + [eos])
                token_ls = tokens[max_len:]
    return token_output_ls

In [95]:
content_input = prep_for_tensor(content_split_tok, 512)
tag_input = prep_for_tensor(tag_split_tok, 512)
word_id_input = prep_for_tensor(word_id_split_tok, 512)

In [132]:
content_split_tok[0]

[807, 6493, 517, 6079, 5141, 3942, 7836, 833]

In [58]:
content_split_tok[1]

[4656]

In [51]:
# def add_padding(token_list, max_len=512):
#     '''
#     INPUT
#         token_list : list
#             e.g. [[1,2,3,4], [5,6,7,8]]
#         max_len : int
#     OUTPUT
#         token_ls : list
#     '''
#     import copy
#     token_ls = copy.deepcopy(token_list)
#     max_len = max_len-2 # to consider the special tokens
#     for index, tokens in enumerate(token_ls):
#         n_token = len(tokens)
#         pad = 1 # an index of special token
#         bos = 2 # an index of special token
#         eos = 3 # an index of special token
#         # add padding if the length is shorter than max_len
#         if n_token < max_len:
#             token_ls[index] += [pad] * (max_len - n_token) # 부족한 만큼 padding을 추가함
        
#         # if a sentence is longer than max_len, cut a sentence.
#         elif n_token > max_len:
#             token_ls[index] = tokens[:max_len]
        
#         # to add begining token and end token
#         token_ls[index] = [bos] + token_ls[index] + [eos]
#     return token_ls

In [52]:
# content_input = add_padding(content_split_tok, 512)
# tag_input = add_padding(tag_split_tok, 512)
# word_id_input = add_padding(word_id_split_tok, 512)

In [53]:
# tag_input[100]

In [96]:
import pickle
with open("./data/content_input_20220531.pickle", "wb") as fw:
    pickle.dump(content_input, fw)
with open("./data/tag_input_20220531.pickle", "wb") as fw:
    pickle.dump(tag_input, fw)
with open("./data/word_id_input_20220531.pickle", "wb") as fw:
    pickle.dump(word_id_input, fw)

# To load data

In [1]:
import torch
import pickle
with open("./data/voca2idex.pickle", "rb") as fr:
    voca2idex = pickle.load(fr)
with open("./data/idex2voca.pickle", "rb") as fr:
    idex2voca = pickle.load(fr)
with open("./data/tag2idex.pickle", "rb") as fr:
    tag2idex = pickle.load(fr)
with open("./data/idex2tag.pickle", "rb") as fr:
    idex2tag = pickle.load(fr)
    
# configuration parameters
VOCA_SIZE = len(voca2idex)
print(VOCA_SIZE)    

In [2]:
import torch
import pickle
with open("./data/content_input_20220531.pickle", "rb") as fr:
    content_input = pickle.load(fr)
with open("./data/tag_input_20220531.pickle", "rb") as fr:
    tag_input = pickle.load(fr)
with open("./data/word_id_input_20220531.pickle", "rb") as fr:
    word_id_input = pickle.load(fr)

# data loader

In [3]:
import torch

In [4]:
input_x = torch.tensor(content_input)
input_y = torch.tensor(tag_input)

In [5]:
input_x[0]

tensor([    2, 13782, 75328, 67477,     0, 13189,     0, 75210, 73263, 13782,
        75328,     0, 75210, 73263, 13782,     0, 73916, 60672,     0, 75210,
        73263, 13782, 75328,     0, 75486, 74865, 74865,     0, 74865, 74397,
        74865, 74397, 74865, 74865, 64762, 68574, 74865, 74865,     0, 74939,
            0, 66726, 66726, 66726, 66726, 35799, 74778, 74778, 74874, 35799,
        74778, 74874, 74778, 75206, 66063, 70938, 68762, 74703, 74623, 75206,
        66063, 70938, 75494, 68762, 74703, 74623, 75206, 66063, 70938, 68762,
        74703, 74623, 75197, 75206, 66063, 70938, 68762, 74703, 74623, 72157,
        74865, 44274,     0, 72157, 74865, 44274, 72157, 75260, 74865, 44274,
        72157, 43623, 74865, 44274,     0, 71938,     0,     0, 44972, 53063,
            0,     0,     0, 44972,     0, 75112,     0,     0,     0, 44972,
            0, 44972, 75112, 75112, 74865, 74865, 74865,     0, 74865, 74517,
        72509,     0, 75256, 72509, 67432, 74517, 72509, 75256, 

In [6]:
input_y[30]

tensor([   2,    5,    6,    6, -100,    6,    6, -100,    6,    6, -100,    6,
           5,    6, -100,    5, -100, -100, -100,    5,    6,    5,    6,    5,
           5,    6,    5,    5,    5, -100,    6,    5,    5,    5,    6,    6,
           5, -100,    6,    6,    5, -100,    6,    6,    5, -100,    6,    6,
           5, -100, -100,    5,    6,    5,    6,    5,    5,    6,    6,    5,
           6,    6,    6,    5,    6,    5,    5,    6,    6,    5,    5,    6,
           5,    5,    6,    6,    6,    5, -100, -100,    6,    6,    5,    5,
        -100, -100,    6,    6,    5,    5, -100, -100,    5,    6,    6,    5,
           5, -100, -100,    6,    6,    6,    6,    6,    5,    6,    6,    6,
           6,    6,    6,    6,    6,    5, -100,    6,    6,    6,    6,    5,
        -100,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
           5,    6,    5, -100,    5,    5, -100,    5,    6,    5, -100,    5,
           6,    5,    5,    5,    6,   

# data

source : https://github.com/naver/nlp-challenge/blob/master/missions/ner/data/train/train_data

In [15]:
import pandas as pd

In [16]:
# data = pd.read_csv('data/train_data.txt', sep='\t',  header=None)
# data.columns = ['index' , 'content', 'tag']
# data['word_id'] = [k for k in range(len(data['content']))]

In [17]:
# data = pd.read_csv('data/indexingFile.csv', sep=',',  header=0)
# data.columns = ['number', 'index' , 'content', 'tag']
# data = data.drop(['number'], axis=1)
# data['word_id'] = [k for k in range(len(data['content']))]

In [18]:
data = pd.read_csv('data/indexingFile_527.csv', sep=',',  header=0)
data.columns = ['number', 'index' , 'content', 'tag']
data = data.drop(['number'], axis=1)
data['word_id'] = [k for k in range(len(data['content']))]

In [19]:
data.head(25)

,index,content,tag,word_id
0,1,강사,1,0
1,2,로,0,1
2,3,활동,0,2
3,4,잘할,0,3
4,5,같은,0,4
5,1,일타,0,5
6,2,강사,1,6
7,3,로,0,7
8,4,같은,0,8
9,1,일타,0,9


To split by a sentence

In [20]:
content_split = []
tag_split = []
word_id_split = []

content_spliter = []
tag_spliter = []
word_spliter = []

for i in range(len(data)):
    content_spliter.append(data['content'].iloc[i])
    tag_spliter.append(data['tag'].iloc[i])
    word_spliter.append(data['word_id'].iloc[i])
    if (i == len(data)-1) : # the last word in a data
        content_split.append(content_spliter)
        content_spliter = []
        tag_split.append(tag_spliter)
        tag_spliter = []
        word_id_split.append(word_spliter)
        word_spliter = []
    elif (data['index'].iloc[i+1] == 1) : # the last word in a sentence
        content_split.append(content_spliter)
        content_spliter = []
        tag_split.append(tag_spliter)
        tag_spliter = []
        word_id_split.append(word_spliter)
        word_spliter = []
        
# sanity check
len(content_split) == len(tag_split) == len(word_id_split)

True

In [21]:
len(content_split)

703464

In [22]:
len(tag_split)

703464

In [23]:
len(word_id_split)

703464

## vocabulary

konlpy tokenizer

In [24]:
from konlpy.tag import Okt
okt = Okt()

print('형태소 분석 :',okt.morphs("자연어 처리 연구를 기반으로 인공지능 모델을 구성하고 있습니다."))
print('품사 태깅 :',okt.pos("자연어 처리 연구를 기반으로 인공지능 모델을 구성하고 있습니다."))
print('명사 추출 :',okt.nouns("자연어 처리 연구를 기반으로 인공지능 모델을 구성하고 있습니다."))

형태소 분석 : ['자연어', '처리', '연구', '를', '기반', '으로', '인공', '지능', '모델', '을', '구성', '하고', '있습니다', '.']
품사 태깅 : [('자연어', 'Noun'), ('처리', 'Noun'), ('연구', 'Noun'), ('를', 'Josa'), ('기반', 'Noun'), ('으로', 'Josa'), ('인공', 'Noun'), ('지능', 'Noun'), ('모델', 'Noun'), ('을', 'Josa'), ('구성', 'Noun'), ('하고', 'Josa'), ('있습니다', 'Adjective'), ('.', 'Punctuation')]
명사 추출 : ['자연어', '처리', '연구', '기반', '인공', '지능', '모델', '구성']


To create the vocabulary corpus

In [25]:
# to select unique value in content_split
unique_content_list = list(set([x for sublist in content_split for x in sublist]))

# to select only string
filtered = filter(lambda voca: isinstance(voca, str), unique_content_list)
unique_filtered_content_list = list(filtered)

# concatenation for tokenizer
unique_filtered_content_str = ' '.join(unique_filtered_content_list)
tokenizer_voca = okt.nouns(unique_filtered_content_str)
print(len(tokenizer_voca))
print(tokenizer_voca[:10])

75559
['세트', '해하', '안슬것', '별식', '좆', '급', '중', '놈', '이기', '개비']


In [26]:
len(okt.nouns(unique_filtered_content_str))

75559

In [27]:
tokenizer_voca = okt.nouns(unique_filtered_content_str)

In [28]:
tokenizer_voca[:10]

['세트', '해하', '안슬것', '별식', '좆', '급', '중', '놈', '이기', '개비']

In [29]:
voca2idex = {gene: idx+5 for idx, gene in enumerate(tokenizer_voca)}
voca2idex['<unk>']= 0 # unknown
voca2idex['<pad>']= 1 # padding
voca2idex['<bos>']= 2 # begining of sentence
voca2idex['<eos>']= 3 # end of sentence
voca2idex['<mask>']= 4 # masking

# configuration parameters
VOCA_SIZE = len(voca2idex)
print(VOCA_SIZE)

# index2voca
idex2voca = {v:k for k,v in voca2idex.items()}
#print(list(gene2idex.keys())[list(gene2idex.values()).index(16)]) 
print(f'{idex2voca[0]}')

21062
<unk>


voca to index

In [30]:
# seq_to_index function
def sequence_to_index(string_list, voca2idex):
    '''
    INPUT
        string_list : string list
    OUTPUT
        integer list
    '''
    converted_index = [voca2idex[word] if word in voca2idex.keys() else voca2idex['<unk>'] for word in string_list]
    return converted_index

# example
print(sequence_to_index(['멋대로', '가나다라마'], voca2idex)) # example

[60453, 0]


## tag integer encodding

In [31]:
tag_list = data['tag'].unique()
num_tags = len(tag_list)
tag_list

array([1, 0])

In [32]:
# tag2idex = {tag: idx for idx, tag in enumerate(tag_list)}
# tag2idex

In [33]:
idex2tag = {}
idex2tag[0]='<unk>'
idex2tag[1]='<pad>'
idex2tag[2]='<bos>'
idex2tag[3]='<eos>'
idex2tag[4]='<mask>'
idex2tag[5]='non'
idex2tag[6]='coffe'

In [34]:
tag2idex = {'<unk>':0,'<pad>':1,'<bos>':2,'<eos>':3,'<mask>':4, 0:5, 1:6}

In [35]:
tag2idex[0]

5

In [37]:
# seq_to_index function
def tag_to_index(sequence, embedding_index):
    '''
    INPUT
        sequence : string list
    OUTPUT
        integer list
    '''
    converted_index = [embedding_index[word] if word in embedding_index.keys() else embedding_index['<unk>'] for word in sequence]
    return converted_index

# # index2gene
# idex2tag = {v:k for k,v in tag2idex.items()}
# idex2tag[0]='[UNK]'
# idex2tag[1]='[PAD]'
# idex2tag[2]='[CLS]'
# idex2tag[3]='[SEP]'
# idex2tag[4]='[MASK]'
# #print(list(gene2idex.keys())[list(gene2idex.values()).index(16)]) 
# print(f'{idex2tag[1]}')

print(tag_to_index(['5', '6'], tag2idex)) # example

[0, 0]


In [ ]:
# tag_to_index(['PER_B', 'DAT_B'], tag2idex)

In [54]:
import pickle
with open("data/voca2idex.pickle", "wb") as fw:
    pickle.dump(voca2idex, fw)
with open("data/idex2voca.pickle", "wb") as fw:
    pickle.dump(idex2voca, fw)    
with open("data/tag2idex.pickle", "wb") as fw:
    pickle.dump(tag2idex, fw)
with open("data/idex2tag.pickle", "wb") as fw:
    pickle.dump(idex2tag, fw)

## tokenizer

In [ ]:
# content_tokenizer = []
# word_id_tokenizer = []
# tag_id_tokenizer = []
# label_ids_tokenizer = []
# for i in data['word_id'][0:50]:
#     # tokenizer
#     content_tokenizer += tokenizer.encode(data['content'].iloc[i])[1:-1]
#     word_id_tokenizer += [i]*(len(tokenizer.encode(data['content'].iloc[i]))-2)    
#     label_ids_tokenizer += [tag2idex[data['tag'].iloc[i]]] + [-100] * (len(tokenizer.encode(data['content'].iloc[i]))-3)
 
    
# data_tokenizer = pd.DataFrame({'token': content_tokenizer, 'word_id': word_id_tokenizer, 'label_ids': label_ids_tokenizer})

In [42]:
# tokenizer
tokenizer = okt.morphs

In [82]:
import numpy as np
import pandas as pd
def tokenizer_per_sentece(content_list, tag_list, word_id_list):
    """
    INPUT:
        content_split_list : a list that represents a sentence 
            e.g. ['비토리오', '양일', '만에', '영사관', '감호', '용퇴,', '항룡', '압력설', '의심만', '가율']
        tag_split_list
            e.g. ['PER_B', 'DAT_B', '-', 'ORG_B', 'CVL_B', '-', '-', '-', '-', '-']
        word_id_split_list
            e.g. [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    OUTPUT: tokenzier results
        content_tokenizer : a tokenized content list without special tokens
        word_id_tokenizer : a word id list
        tag_id_tokenizer : a tag id list that the only first content has a tag.
    """
    # sanity check
    if not (len(content_list)==len(tag_list)==len(word_id_list)):
        print("check the input data")
    
    # tokenizer
    content_tokenizer = []
    word_id_tokenizer = []
    tag_id_tokenizer = []
    
    # error handling : nan in input
    if any(pd.isna(content_list)):
        return [], [], []
    
    for i in range(len(word_id_list)):
        # tokenizer
        content_tokenizer += tokenizer(content_list[i]) # to remove special tokens [CLS] and [SEP]
        word_id_tokenizer += [word_id_list[i]] * (len(tokenizer(content_list[i])))
        tag_id_tokenizer += [tag2idex[tag_list[i]]] + [-100] * (len(tokenizer(content_list[i]))-1)
    
    # to convert string into index
    content_tokenizer = sequence_to_index(content_tokenizer, voca2idex)
    
    # error handling : tokenizer does not work properly
    if not (len(content_tokenizer) == len(word_id_tokenizer) == len(tag_id_tokenizer)):
        # tokenzier does not work properly becuase of noise.
        print(f'sanity check : {len(content_tokenizer) == len(word_id_tokenizer) == len(tag_id_tokenizer)}')
        content_tokenizer = []
        word_id_tokenizer = []
        tag_id_tokenizer = []
    return content_tokenizer, tag_id_tokenizer, word_id_tokenizer

In [61]:
content_split[0]

['강사', '로', '활동', '잘할', '같은']

In [83]:
a, b, c = tokenizer_per_sentece(content_split[0], tag_split[0], word_id_split[0])

In [84]:
a

[13782, 75328, 67477, 0, 13189, 0]

In [90]:
[idex2voca[word] for word in a]

['강사', '로', '활동', '<unk>', '할', '<unk>']

In [85]:
b

[6, 5, 5, 5, -100, 5]

In [86]:
c

[0, 1, 2, 3, 3, 4]

error case

In [39]:
content_split[172999]

['\ufeff', '가장', '한', '추출', '콜드', '추출', '에서', '카페인', '변화']

In [40]:
tag_split[172999]

[0, 0, 0, 1, 1, 1, 0, 1, 0]

In [41]:
word_id_split[172999]

[564651, 564652, 564653, 564654, 564655, 564656, 564657, 564658, 564659]

In [42]:
a, b, c = tokenizer_per_sentece(content_split[172999], tag_split[172999], word_id_split[172999])

sanity check : False


In [43]:
a

[]

Error case

In [66]:
content_split[500120]

[nan]

In [67]:
import numpy as np
np.isnan(content_split[500120])

array([ True])

In [68]:
np.isnan(content_split[500120])

array([ True])

In [69]:
type(content_split[500120][0])

float

In [70]:
a, b, c = tokenizer_per_sentece(content_split[500120], tag_split[500120], word_id_split[500120])

In [71]:
a

[]

error

In [72]:
content_split[598602]

['1', '1', 'coffee', nan, 'Mp']

In [73]:
pd.isna(content_split[598602])

array([False, False, False,  True, False])

In [74]:
any(pd.isna(content_split[598602]))

True

In [75]:
a, b, c = tokenizer_per_sentece(content_split[598602], tag_split[598602], word_id_split[598602])

In [91]:
from tqdm import tqdm 
def tokenizer_all_data(content_split, tag_split, word_id_split):
    # sanity check
    if not (len(content_split)==len(tag_split)==len(word_id_split)):
        print("check the input data")
        
    content_split_token, tag_split_token, word_id_split_token = [], [], []
    for i in tqdm(range(len((content_split)))):
        #print(i)
        content, tag, wordID = tokenizer_per_sentece(content_split[i], tag_split[i], word_id_split[i])
        content_split_token.append(content)
        tag_split_token.append(tag)
        word_id_split_token.append(wordID)
    return content_split_token, tag_split_token, word_id_split_token

In [92]:
content_split_tok, tag_split_tok, word_id_split_tok = tokenizer_all_data(content_split, tag_split, word_id_split)

100%|██████████████████████████████████| 703464/703464 [59:20<00:00, 197.58it/s]


record
- 20220527 : KoBert
- 20220531 : Bert

In [93]:
import pickle
with open("data/content_split_tok_20220531.pickle", "wb") as fw:
    pickle.dump(content_split_tok, fw)
with open("data/tag_split_tok_20220531.pickle", "wb") as fw:
    pickle.dump(tag_split_tok, fw)
with open("data/word_id_split_tok_20220531.pickle", "wb") as fw:
    pickle.dump(word_id_split_tok, fw)

In [65]:
import torch
import pickle
with open("data/content_split_tok_20220531.pickle", "rb") as fr:
    content_split_tok = pickle.load(fr)
with open("data/tag_split_tok_20220531.pickle", "rb") as fr:
    tag_split_tok = pickle.load(fr)
with open("data/word_id_split_tok_20220531.pickle", "rb") as fr:
    word_id_split_tok = pickle.load(fr)

In [53]:
#example
[tokenizer.decode(token) for token in content_split_tok[0]]

['강', '사', '하면', '잘', '할', '스타', '는', '?', '.']

Padding

In [54]:
print(max([len(x) for x in content_split_tok]))
print(sum([len(x) for x in content_split_tok])/len(content_split_tok))

653
6.924360502429632


In [94]:
def prep_for_tensor(token_list, max_len=512):
    '''
    This function get a list of token lists and converts a input list into a list in size of max_len including special tokens.
    
    INPUT
        token_list : a list of token lists
            e.g. [[1,2,3,4], [5,6,7,8], [9, 10, 11, 12, 13, 14, 15]]
        max_len : int
    OUTPUT
        token_ls : a list of lists in size of max_len with special tokens. The last token that does not fit the size would be dropped.
            e.g. [[0, 1, 2, 3, 4, 0], [0, 5, 6, 7, 8, 0], [0, 9, 10, 11, 12, 0]]
    '''
    # setting
    max_len = max_len-2 # to consider the special tokens
    pad = 1 # an index of special token
    bos = 2 # an index of special token
    eos = 3 # an index of special token
    token_output_ls = []
    token_ls = []
    #token_ls_iter_position = 0 # index of the last element in token_ls
    
    for index, tokens in enumerate(token_list):
        n_token = len(tokens)
        
        if len(token_ls) + n_token <= max_len :
            token_ls += tokens
            #token_ls_iter_position += len(token_ls)
        else : 
            cut_position = max_len - len(token_ls)
            #assert cut_position >= 0
            token_ls += tokens[:cut_position]

            # to collect the current token_ls
            token_ls = [bos] + token_ls + [eos] # to add special tokens

            if not len(token_ls)-2 == max_len : 
                print(f'index: {index}')
                print(f'cut_position: {cut_position}')
                print(len(token_ls))
            assert len(token_ls)-2 == max_len
            token_output_ls.append(token_ls)

            # to update token_ls for next iteration
            token_ls = tokens[cut_position:]
            #token_ls_iter_position = len(token_ls) - 1
            
            # if token_ls is longer than max_len, it requires an additional cleavage
            while (len(token_ls) > max_len):
                token_output_ls.append([bos] + token_ls[:max_len] + [eos])
                token_ls = tokens[max_len:]
    return token_output_ls

In [95]:
content_input = prep_for_tensor(content_split_tok, 512)
tag_input = prep_for_tensor(tag_split_tok, 512)
word_id_input = prep_for_tensor(word_id_split_tok, 512)

In [132]:
content_split_tok[0]

[807, 6493, 517, 6079, 5141, 3942, 7836, 833]

In [58]:
content_split_tok[1]

[4656]

In [51]:
# def add_padding(token_list, max_len=512):
#     '''
#     INPUT
#         token_list : list
#             e.g. [[1,2,3,4], [5,6,7,8]]
#         max_len : int
#     OUTPUT
#         token_ls : list
#     '''
#     import copy
#     token_ls = copy.deepcopy(token_list)
#     max_len = max_len-2 # to consider the special tokens
#     for index, tokens in enumerate(token_ls):
#         n_token = len(tokens)
#         pad = 1 # an index of special token
#         bos = 2 # an index of special token
#         eos = 3 # an index of special token
#         # add padding if the length is shorter than max_len
#         if n_token < max_len:
#             token_ls[index] += [pad] * (max_len - n_token) # 부족한 만큼 padding을 추가함
        
#         # if a sentence is longer than max_len, cut a sentence.
#         elif n_token > max_len:
#             token_ls[index] = tokens[:max_len]
        
#         # to add begining token and end token
#         token_ls[index] = [bos] + token_ls[index] + [eos]
#     return token_ls

In [52]:
# content_input = add_padding(content_split_tok, 512)
# tag_input = add_padding(tag_split_tok, 512)
# word_id_input = add_padding(word_id_split_tok, 512)

In [53]:
# tag_input[100]

In [96]:
import pickle
with open("./data/content_input_20220531.pickle", "wb") as fw:
    pickle.dump(content_input, fw)
with open("./data/tag_input_20220531.pickle", "wb") as fw:
    pickle.dump(tag_input, fw)
with open("./data/word_id_input_20220531.pickle", "wb") as fw:
    pickle.dump(word_id_input, fw)

# To load data

In [1]:
import torch
import pickle
with open("./data/voca2idex.pickle", "rb") as fr:
    voca2idex = pickle.load(fr)
with open("./data/idex2voca.pickle", "rb") as fr:
    idex2voca = pickle.load(fr)
with open("./data/tag2idex.pickle", "rb") as fr:
    tag2idex = pickle.load(fr)
with open("./data/idex2tag.pickle", "rb") as fr:
    idex2tag = pickle.load(fr)
    
# configuration parameters
VOCA_SIZE = len(voca2idex)
print(VOCA_SIZE)    

In [2]:
import torch
import pickle
with open("./data/content_input_20220531.pickle", "rb") as fr:
    content_input = pickle.load(fr)
with open("./data/tag_input_20220531.pickle", "rb") as fr:
    tag_input = pickle.load(fr)
with open("./data/word_id_input_20220531.pickle", "rb") as fr:
    word_id_input = pickle.load(fr)

# data loader

In [3]:
import torch

In [4]:
input_x = torch.tensor(content_input)
input_y = torch.tensor(tag_input)

In [5]:
input_x[0]

tensor([    2, 13782, 75328, 67477,     0, 13189,     0, 75210, 73263, 13782,
        75328,     0, 75210, 73263, 13782,     0, 73916, 60672,     0, 75210,
        73263, 13782, 75328,     0, 75486, 74865, 74865,     0, 74865, 74397,
        74865, 74397, 74865, 74865, 64762, 68574, 74865, 74865,     0, 74939,
            0, 66726, 66726, 66726, 66726, 35799, 74778, 74778, 74874, 35799,
        74778, 74874, 74778, 75206, 66063, 70938, 68762, 74703, 74623, 75206,
        66063, 70938, 75494, 68762, 74703, 74623, 75206, 66063, 70938, 68762,
        74703, 74623, 75197, 75206, 66063, 70938, 68762, 74703, 74623, 72157,
        74865, 44274,     0, 72157, 74865, 44274, 72157, 75260, 74865, 44274,
        72157, 43623, 74865, 44274,     0, 71938,     0,     0, 44972, 53063,
            0,     0,     0, 44972,     0, 75112,     0,     0,     0, 44972,
            0, 44972, 75112, 75112, 74865, 74865, 74865,     0, 74865, 74517,
        72509,     0, 75256, 72509, 67432, 74517, 72509, 75256, 

In [6]:
input_y[30]

tensor([   2,    5,    6,    6, -100,    6,    6, -100,    6,    6, -100,    6,
           5,    6, -100,    5, -100, -100, -100,    5,    6,    5,    6,    5,
           5,    6,    5,    5,    5, -100,    6,    5,    5,    5,    6,    6,
           5, -100,    6,    6,    5, -100,    6,    6,    5, -100,    6,    6,
           5, -100, -100,    5,    6,    5,    6,    5,    5,    6,    6,    5,
           6,    6,    6,    5,    6,    5,    5,    6,    6,    5,    5,    6,
           5,    5,    6,    6,    6,    5, -100, -100,    6,    6,    5,    5,
        -100, -100,    6,    6,    5,    5, -100, -100,    5,    6,    6,    5,
           5, -100, -100,    6,    6,    6,    6,    6,    5,    6,    6,    6,
           6,    6,    6,    6,    6,    5, -100,    6,    6,    6,    6,    5,
        -100,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
           5,    6,    5, -100,    5,    5, -100,    5,    6,    5, -100,    5,
           6,    5,    5,    5,    6,   

In [7]:
from torch.utils.data import TensorDataset 
from torch.utils.data import DataLoader
from torch.utils.data import random_split

batch_size=8
total_dataset = TensorDataset(input_x, input_y)

num_train = int(len(total_dataset)*0.8)
num_validation = int(len(total_dataset)*0.1)
num_test = len(total_dataset)-num_train-num_validation

train_dataset, validation_dataset, test_dataset = random_split(total_dataset, [num_train, num_validation, num_test])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


print(len(total_dataset))
print(len(train_dataset))
print(len(validation_dataset))
print(len(test_dataset))
print('sanity check: ', len(total_dataset)==len(train_dataset)+len(validation_dataset)+len(test_dataset))


len(train_loader)
len(validation_loader)
len(test_loader)

del total_dataset, num_train, num_validation, num_test, train_dataset, validation_dataset, test_dataset

6064
4851
606
607
sanity check:  True
